In [1]:
from transformer_proto5 import *

C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis


In [2]:
# load and preprocess data
direction = ['left','right','front']
files = []

for d in direction:
    files.extend(glob.glob(f"G:/UAL_Thesis/affective_computing_datasets/multiview-emotional-expressions-dataset/*/{d}_*/processed_data.json"))

processed_data = preprocess_data(files)

x_list = processed_data['x']
y_list = processed_data['y']
dx_list = processed_data['dx']
dy_list = processed_data['dy']
conf_list = processed_data['confidence']
emotions_labels = processed_data['emotions']

# prepare data for training - deltas from now on
global max_x, min_x, max_y, min_y

max_dx, min_dx, normalised_dx = normalize_values_2D(dx_list)
max_dy, min_dy, normalised_dy = normalize_values_2D(dy_list)
max_x, min_x, normalised_x = normalize_values_2D(dx_list)
max_y, min_y, normalised_y = normalize_values_2D(dy_list)

dkp_frames = create_kp_frames(normalised_dx, normalised_dy)  # 1D tensor array of 50 numbers (x,y,x,y --> 25 keypoints)
kp_frames = create_kp_frames(normalised_x, normalised_y)  # 1D tensor array of 50 numbers (x,y,x,y --> 25 keypoints) - for getting first frames/validation

data = add_delta_to_frames(kp_frames, dkp_frames)
data = add_emotions_to_frames(data, emotion_labels_to_vectors(emotions_labels))


frame_dim = len(data[0][0]) # how many numbers are in each frame? - 50 kps xy + 50 deltas + 7 emotion 
print(f"frame_dim: {frame_dim}")
train_data, val_data = stratified_split(data, test_size=0.1)


# HYPERPARAMETERS------------------

torch.manual_seed(1337)
BATCH_SIZE = 4 # how many independent sequences will we process in parallel? - every forward and backward pass in transformer
BLOCK_SIZE = 16 # what is the maximum context length for predictions? 
DROPOUT = 0.3
LEARNING_RATE = 0.0001
EPOCHS = 1000
FRAMES_GENERATE = 300
TRAIN = False
EVAL_EVERY = 100
CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"
L1_LAMBDA = None
L2_REG=0.0
global train_seed

# ---------------------------------

# NOTES---------------------------------
notes = f"""Got rid of both L1 and L2, increasing dropout because model acting weird, this is now delta"""
# ---------------------------------

# create model

m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization

# train
if TRAIN:
    # Generate a random seed
    
    train_seed = random.randint(1, 100000)
    print(f'Training model {train_seed}...')
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')  # Initialize with infinity, so first instance is saved

    
    for epoch in tqdm(range(EPOCHS), desc="Training", unit="epoch"):
        # get sample batch of data
        xb,yb,mask = get_batch('train',BLOCK_SIZE,BATCH_SIZE)
        # validate emotions are consistent
        validate_emotion_consistency(xb, yb)
        # evaluate loss
        logits, loss = m(xb,yb,l1_lambda=L1_LAMBDA)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        
        if epoch % EVAL_EVERY == 0:
            losses = estimate_loss()
            print(f"\nTrain loss: {losses['train']:.6f} val loss: {losses['val']:.6f}")
            
            if (epoch != 0):
                # Store the losses for plotting
                train_losses.append(losses['train'])
                val_losses.append(losses['val'])
            
            if (epoch % (EVAL_EVERY*10) == 0) and (epoch != 0):
                # Save a checkpoint every 10 rounds of eval if it has the best validation loss so far
                if losses['val'] < best_val_loss:
                    print(f"-> Best model so far (val loss: {best_val_loss:.6f}), saving model...")
                    best_val_loss = losses['val']
                    save_checkpoint(model=m, optimizer=optimizer, epoch=epoch, loss=loss, checkpoint_path=CHECKPOINT_PATH)
                    print(f'Model {train_seed} saved!')
                    
    # After the training loop, save the final model
    if val_losses[-1] < best_val_loss:
                    print(f"-> Best model so far (val loss: {best_val_loss:.6f}), saving model...")
                    best_val_loss = val_losses[-1]
                    save_checkpoint(model=m, optimizer=optimizer, epoch=EPOCHS, loss=val_losses[-1], checkpoint_path=CHECKPOINT_PATH)
                    print(f'Model {train_seed} saved!')
    # After the training loop, plot the losses
    plot_losses(train_losses, val_losses)
    
else:
    # Load the model
    print('Loading model...')
    m, optimizer, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)
    print(f"Model {train_seed} loaded from {CHECKPOINT_PATH} (epoch {epoch}, loss {loss:.6f})")

# Generate a sequence
print(f'Generating sequence of {FRAMES_GENERATE} frames...')
xb,yb,mask = get_batch('val',BLOCK_SIZE,BATCH_SIZE)

generated = m.generate(xb, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_dx, min_dx, max_dy, min_dy)



100%|██████████| 4102/4102 [00:03<00:00, 1059.42it/s]


Validating interpolation...
No errors found!
Generating deltas...


100%|██████████| 4102/4102 [00:06<00:00, 627.19it/s]


In [ ]:
    # # visualise and save
    # for batch in unnorm_out:
    #     visualise_skeleton(batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}')
    
    # if TRAIN:
    #     write_notes(notes)
     
    # print('Done!')